In [ ]:
import sys 
import torch
import torch.nn as nn
import torch.optim as optim
from utils import *
import matplotlib.pyplot as plt

if '..' not in sys.path:
    sys.path.append('../')

folder_path = '/Users/antongolles/Documents/uni/masters/myMasters/applied_machine_learning/inertial_navigation_transformer/data/data_from_RoNIN/train_dataset_1/'

# Define constant variables
FOLDER_PATH = '/Users/antongolles/Documents/uni/masters/myMasters/applied_machine_learning/inertial_navigation_transformer/data/data_from_RoNIN/train_dataset_1/'

N_TRAIN = 60_000
N_VAL_FRACTION = .2
N_TEST = 25_000
NUM_DATASETS = 10
SEQ_LEN = 100
BATCH_SIZE = 1
LEARNING_RATE = 0.003
NUM_EPOCHS = 10

# Set input and output features
INPUT_FEATURES = ['synced/gyro', 'synced/acce', 'synced/magnet']
OUTPUT_FEATURES = ['pose/tango_pos']

params = {'N_train': N_TRAIN, 
        'N_test': N_TEST,
        'seq_len': SEQ_LEN, 
        'input': INPUT_FEATURES,
        'output': OUTPUT_FEATURES, 
        'normalize': False,
        'verbose': False, 
        'num_datasets': NUM_DATASETS,
        'random_start': True,
        'overlap': 0,
        'include_theta': False}

X_train, y_train, X_test, y_test, col_locations = load_split_data(folder_path=FOLDER_PATH, **params)

print('X_train shape: ', X_train.shape, '\ny_train shape: ', y_train.shape, '\nX_test shape: ', X_test.shape, '\ny_test shape: ', y_test.shape)

def preprocess_data(X_train, y_train, X_test, y_test, col_locations):
    # for y, subtract the first value from all values

# preprocess data
X_train, y_train, X_test, y_test = preprocess_data(X_train, y_train, X_test, y_test, col_locations)


dataset_train = torch.utils.data.TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
dataloader = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=False)



input_features = ['synced/gyro']
output_features = ['pose/tango_pos']
if include_acc:
    input_features.append('synced/acce')
if include_mag:
    input_features.append('synced/magnet')
if include_lin_acc:
    input_features.append('synced/linacce')
if include_ori:
    output_features.append('pose/tango_ori')



params = {'N_train': Ntrain, 
        'N_test': Ntest,
        'seq_len': seq_len, 
        'input': input_features,
        'output': output_features, 
        'normalize': normalize,
        'verbose': False, 
        'num_datasets':num_datasets,
        'random_start':random_start,
        'overlap': overlap,
        'include_theta': include_theta,}
X_train, y_train, X_test, y_test, col_locations = load_split_data(folder_path=folder_path, **params)
print(col_locations)
print(X_train.shape)


d_model = 7
nhead = 2
dim_feedforward = 120
learning_rate = 0.003
num_epochs = 10
hidden_size = 64
batch_size = 1
num_layers = 3

# dataset
dataset_train = torch.utils.data.TensorDataset( torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float() )
# dataloader
dataloader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=False)


# model
# set up a tansformer model

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, batch_size, dropout=0.1):
        super().__init__()
        
        

        # Config variables
        self.d_model = 7
        self.nhead = 1
        self.num_layers = num_layers
        self.dim_feedforward = hidden_size * 4

        self.fc_get_next_quat = nn.Sequential(
            nn.Linear(4*4 + 4, hidden_size),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.Linear(hidden_size, 4)
        )

        self.quat_cleaner = nn.Sequential(
            nn.Linear(13, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size*2),
            nn.ReLU(),
            
            nn.ReLU(),
            nn.Linear(hidden_size*2, hidden_size),
            #nn.ReLU(),
            nn.Linear(hidden_size, 4)
        )

        #self.conv1 = nn.Conv1d(self.d_model, self.d_model, kernel_size=3, padding=1)

        self.transformer = nn.Transformer(d_model=self.d_model, nhead=self.nhead, num_encoder_layers=self.num_layers, num_decoder_layers=self.num_layers, dim_feedforward=self.dim_feedforward, dropout=dropout, batch_first=True)

        self.fc_out = nn.Sequential(
            nn.Linear(7, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size*2),
            #nn.ReLU(),
            #nn.Linear(hidden_size*2, hidden_size*4),
            #nn.ReLU(),
            #nn.Linear(hidden_size*4, hidden_size*4),
            #nn.ReLU(),
            #nn.Linear(hidden_size*4, hidden_size*2),
            #nn.ReLU(),
            nn.Linear(hidden_size*2, hidden_size),
            #nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

        # set all the weights to be the same
        # for p in self.parameters():
        #     if p.dim() > 1:
        #         nn.init.xavier_uniform_(p)
        # #set all the weights to be 0
        # for p in self.parameters():
        #     p.data.fill_(0)
            

    def forward(self, x):
        # {'input': {'synced/gyro': (0, 3), 'synced/acce': (3, 6), 'synced/magnet': (6, 9), 'theta': (9, 25)}, 
        gyro =  x[:,:,0:3]; theta = x[:,:,9:25]
        magnet = x[:,:,3:6]
        acce = x[:,:,6:9]
        
        # get initial orientation
        R = [get_body2world_rot(m[0].numpy(),a[0].numpy()) for m,a in zip(magnet, acce)]
        quat_start = [rotation_matrix_2_quaternion(np.linalg.inv(r)) for r in R]
        quat_start = torch.tensor(quat_start).float().squeeze().unsqueeze(0)
        quats = [quat_start]
        # get all the quaternions
        for i in range(1, len(theta[0])):
            quat = quats[i-1].squeeze()
            theta_ = theta.squeeze()[i-1:i]#
            quat_next_ana = theta_.reshape(4,4) @ quat
            catted = torch.cat((quat_next_ana, theta_.squeeze()))
            next_quat = self.fc_get_next_quat(catted)
            next_quat = next_quat / torch.norm(next_quat)
            quats.append(next_quat.reshape(1,4) )
        quats = torch.stack(quats).squeeze().unsqueeze(0)
        

        # clean quats 
        #print('quat shape', quats.shape, 'gyro shape', gyro.shape, 'acce shape', acce.shape, 'magnet shape', magnet.shape)
        quats = self.quat_cleaner(torch.cat((quats, gyro, acce, magnet), dim=2))
        quats = quats / torch.norm(quats, dim=1)
        quats = quats.squeeze()
        # rotate the acce and magnet
        #print(acce.shape)
        acce_and_quat = torch.cat((acce.squeeze(), quats), dim=-1).unsqueeze(0)
        #print(acce_and_quat.shape)
        trans_out = self.transformer(acce_and_quat, acce_and_quat)
        #print(trans_out.shape)
        
        final_out = self.fc_out(trans_out)
        #print(final_out.shape)
        # select first 3 elements
        final_out = final_out[:,:,0:3]
        return final_out

# Training Loop
net = Net(input_size=X_train.shape[2], hidden_size=hidden_size, output_size=7, num_layers=num_layers, batch_size=batch_size)
optim_ = optim.AdamW(params=net.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optim_, step_size=0.1, gamma=0.05) # Add a learning rate scheduler
criterion = nn.MSELoss()
losses = []
from tqdm import tqdm
for epoch in range(num_epochs):
    running_loss = 0.0
    for data in tqdm(dataloader):
        X_, y_ = data
        optim_.zero_grad()
        output =  net(X_)
        loss = criterion(output, y_)
        loss.backward()
        #nn.utils.clip_grad_norm_(net.parameters(), max_norm=10)  # Uncommented gradient clipping
        optim_.step()
        scheduler.step()  # Added step for learning rate scheduler
        running_loss += loss.item()
        #print(loss.item() / batch_size)
    print(f'epoch: {epoch}, loss: {running_loss}')
    losses.append(running_loss)


# plot the losses
plt.plot(losses)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.yscale('log')
short_name = 'tiny'
plt.savefig(f'../figures/{short_name}_transformer_{d_model}_{nhead}_{num_layers}_{dim_feedforward}.png', dpi=300)

# test
N_test = torch.from_numpy(X_test[:1]).float()

# set the model to evaluation mode
net.eval()

# make predictions
with torch.no_grad():
    predictions = net(N_test)

predictions = predictions.numpy()
accumulated_preds = []
for pred in predictions:
    accumulated_preds.append(pred + accumulated_preds[-1][-1] if accumulated_preds else pred)
predictions = np.array(accumulated_preds)
predictions = predictions.reshape(-1, 3)
# plot the predictions
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
ax[0].plot(predictions[:,0], predictions[:,1], label='predictions')
ax[0].legend()
y_test_reshaped = y_test.reshape(-1, 3)
y_test_reshaped -= y_test_reshaped[0]
ax[1].plot(y_test_reshaped[:,0], y_test_reshaped[:,1], label='truth')
ax[1].legend()

# save the model
import datetime
timestamp_ = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
torch.save(net.state_dict(), f'../transformer_big_{timestamp_}.pt')